# Imports,  Settings and Paths

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
from Code.Models.c7o2h10_model import DeepPotential, train

In [3]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
model_path = './ModelCheckpoints/c7o2h10/ '

# Read the Data

In [4]:
X_train_npy = np.load(data_path)
Y_train_npy = np.load(label_path) * -1
X_train_npy.shape

(404000, 19, 72)

### Check for NaN values

In [5]:
use_ids = np.where(np.isnan(X_train_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_train_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [6]:
X_train_npy = X_train_npy[use_ids]
Y_train_npy = Y_train_npy[use_ids]
X_train_npy.shape

(404000, 19, 72)

### Shuffle the Dataset

In [7]:
shuffle = np.arange(X_train_npy.shape[0])
np.random.shuffle(shuffle)
X_train_npy = X_train_npy[shuffle]
Y_train_npy = Y_train_npy[shuffle]

### Normalize Dataset

In [8]:
#Y_min = Y_train_npy.min()
#Y_train_npy -= Y_min
#Y_max = Y_train_npy.max()
#Y_train_npy /= Y_max

In [9]:
#print('Mean: {}\nVariance: {}'.format(Y_train_npy.mean(), Y_train_npy.var()))

### Create Pytorch Variables

In [10]:
X_data = Variable(torch.Tensor(X_train_npy))
Y_data = Variable(torch.Tensor(Y_train_npy), requires_grad=False) - 11500

# Train Model

## Do the learning

In [11]:
init_deep_pot = DeepPotential()
init_optim = torch.optim.Adam(init_deep_pot.parameters(), lr=1e-2)
deep_pot, optim = train(init_deep_pot, init_optim, X_data, Y_data, 300, 128, (0.01, 0.96, 1.5), print_every=100)


Epoch: 0	learning rate: 0.01
---
total: 0.0 %	current epoch: 4.0 %	loss: 0.832924	time estimate: 4256.1 min
total: 0.0 %	current epoch: 7.9 %	loss: 0.527784	time estimate: 4221.6 min
total: 0.0 %	current epoch: 11.9 %	loss: 0.437198	time estimate: 4212.2 min
total: 0.1 %	current epoch: 15.8 %	loss: 0.394763	time estimate: 4203.4 min
total: 0.1 %	current epoch: 19.8 %	loss: 0.394369	time estimate: 4203.1 min
total: 0.1 %	current epoch: 23.8 %	loss: 0.3515	time estimate: 4207.6 min
total: 0.1 %	current epoch: 27.7 %	loss: 0.33286	time estimate: 4334.3 min
total: 0.1 %	current epoch: 31.7 %	loss: 0.331756	time estimate: 4448.5 min
total: 0.1 %	current epoch: 35.7 %	loss: 0.31955	time estimate: 4506.6 min
total: 0.1 %	current epoch: 39.6 %	loss: 0.298539	time estimate: 4575.7 min
total: 0.1 %	current epoch: 43.6 %	loss: 0.301377	time estimate: 4596.3 min
total: 0.2 %	current epoch: 47.5 %	loss: 0.280944	time estimate: 4612.1 min
total: 0.2 %	current epoch: 51.5 %	loss: 0.277654	time estim

KeyboardInterrupt: 

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)